Cristian Villazhannay <br>
Robotic Manipulation <br>
November 19, 2023 <br>

### Solution Summary 
My solution recognizes that the trajectory generating components are already present through the functions `ScrewTrajectory` and `CartestianTrajectory` in the `modern_robotics` software package. <br>

Therefore, all we need to calculate are the start and end positions used for each of the eight trajectory segments, as defined in the project statement.<br>
<br>

So, the first step in the code will be place everything in the reference frame - since some of our inputs are defined relative to the cuboid frame.<br>
<br>

Then, we use `ScrewTrajectory` to move the gripper frame from the start frame to the next frame defined in the problem statement (grasp, standoff, etc.). Once this is done, we use the helper function `csVector` to convert the `ScrewTrajectory` output into a 13-vector for conversion into a .CSV file. <br>
<br>

For each subsequent frame, we use the last position of the trajectory segment as our starting point. <br>
<br>

Some extra features that I implemented are `method` and `time` which allow the user to implement the total time that the trajectory is going to take, and to define whether trajectory calculation will use cubic or quintic polynomials. 

### Code Execution
To run this code, simply run all the cells and a .CSV file will appear in the directory that the .ipynb file is in. The actual call is in the last cell and is outlined as follows:

`snip = TrajectoryGenerator(T_se, T_isc, T_dsc, T_gce, T_sce, 64)`


In [3]:
#Import the necessary packages. 
import numpy as np 
import modern_robotics as mr
from math import sin,cos

In [4]:
def csVector(traj, grip_state):
    '''
    csVector takes in a trajectory list as output from the ScrewTrajectory or
    Cartestian Trajectory in modern_robotics. It then converts it into a 13 vector
    of the form

    T_ se = [[r_11, r_12, r_13, p_x],
             [r_21, r_22, r_23, p_y],
             [r_31, r_32, r_33, p_z],
             [   0,    0,    0,   1]]

    [r_11, r_12, r_13, r_21, r_22, r_23, r_31, r_32, r_33, p_x, p_y, p_z, grip_state] 

    Parameters
    ----------
    traj : list
        a N shaped list, where each entry is a 4 x 4 configuration state. 

    grip_state: int
        either 0 or 1, relays whether the gripper is open or not. 

    Output
    ------
    vec : np.array
        a (N, 13) shaped array that represents the configuration states 
        and the gripper state.
    '''
    #convert the list into a numpy array.
    trajArr = np.asarray(traj)

    #create an empty list that we will initialize into. 
    vec = np.empty((trajArr.shape[0], 13))

    #Create the 13-vector by flattening the rotation matrix and appending the position at the end. 
    #grip state is called at the end. 
    for i in range(trajArr.shape[0]):
        trans = np.append(np.copy(trajArr[i,0:3,0:3].flatten()), trajArr[i,0:3,-1])
        vec[i,:] = np.append(trans, grip_state)
    
    return vec

In [5]:
def TrajectoryGenerator(T_se, initial_cube, desired_cube, grasp, standoff, time, k = 1, method= "cubic"): 
    '''
    TrajectoryGenerator will generate the reference trajectory for the end effector frame {e}.
    
    Parameters
    -----------
    T_se            : np.array
        A (4,4) shaped array representing the initial configuration of the end-effector
        in the reference trajectory 

    intial_cube     : np.array
        A (4,4) shaped array representing the cube's initial configuration. 

    desired_cube    : np.array
        A (4,4) shaped array representing the cube's final configuration

    grasp           : np.array
        A (4,4) shaped array representing the end-effectors configuration relative to the cube
        when it is grasping the cube

    standoff        : np.array
        A (4,4) shaped array representing the configuration of the end effector relative to the cube. 

    k               : int 
        Must be an integer that is greater than 1. Will default to 1 if not input. 

    time            : int
        Total time in seconds for the trajectory to move. 

    method          : str
        String input that declares what order polynomial we will be using. Will default to cubic. 

    Outputs
    --------
    trajectory      : np.array
        An (N,13) shaped array that represents the N configurations along the entire concatenated eight
        segment reference trajectory. 
    '''

    ##INPUT VARIABLE CHECKS
    #Make sure that k meets the req. 
    if type(k) != int or k < 1:
        raise ValueError("k can only be a integer that is >= 1")
    
    #Make sure that our method will be cubic or quintic. 
    match method: 
        case "cubic":
            method = 3
        case "quintic":
            method = 5
        case _: 
            raise ValueError("method must be cubic or quintic.")

    ##TRAJECTORY INPUT GENERATION
    #Calculate the time that each step will take.
    timeSlice = time/8

    #Configuration of the standoff/grasp configurations in the reference frame. 
    initialsT_se = initial_cube @ standoff
    initialgT_se = initial_cube @ grasp

    desiredsT_se = desired_cube @ standoff
    desiredgT_se = desired_cube @ grasp


    ##TRAJECTORY GENERATION
    #{1} Move the gripper from the initial configuration to the standoff
    trajOne = mr.ScrewTrajectory(T_se, initialsT_se,timeSlice, 100, method)
    csvSnip = csVector(trajOne, 0)

    #{2} Move the gripper from the standoff to the grasp.
    trajTwo = mr.ScrewTrajectory(trajOne[-1], initialgT_se,timeSlice, 100, method)
    csvSnip = np.vstack((csvSnip,csVector(trajTwo, 0)))

    #{3} Close the gripper
    trajThree = mr.ScrewTrajectory(trajTwo[-1], trajTwo[-1],timeSlice, 100, method)
    csvSnip = np.vstack((csvSnip,csVector(trajThree, 1)))

    #{4} A trajectory to move the gripper back up to the "standoff" configuration
    trajFour = mr.ScrewTrajectory(trajThree[-1], initialsT_se,timeSlice, 100, method)
    csvSnip = np.vstack((csvSnip,csVector(trajFour, 1)))

    #{5} A trajectroy to move the gripper to the final configuration "standoff"
    trajFive = mr.ScrewTrajectory(trajFour[-1], desiredsT_se,timeSlice, 100, method)
    csvSnip = np.vstack((csvSnip,csVector(trajFive, 1)))

    #{6} A trajectory to move the gripper to the final configuration
    trajSix = mr.ScrewTrajectory(trajFive[-1], desiredgT_se,timeSlice, 100, method)
    csvSnip = np.vstack((csvSnip,csVector(trajSix, 1)))

    #{7} Open the gripper
    trajSeven = mr.ScrewTrajectory(trajSix[-1], desiredgT_se,timeSlice, 100, method)
    csvSnip = np.vstack((csvSnip,csVector(trajSeven, 0)))

    #{8} Move back to the standoff. 
    trajEight = mr.ScrewTrajectory(trajSeven[-1], desiredsT_se,timeSlice, 100, method)
    csvSnip = np.vstack((csvSnip,csVector(trajEight, 0)))

    ##CSV CREATION
    #Compile our trajectory.
    np.savetxt('theta' + '.csv', csvSnip, delimiter=',')
    return csvSnip


In [10]:
#Initial configuration of the end effector reference trajectory
T_se = np.array([[ 1, 0, 0, 0.199],
                 [ 0, 1, 0,     0],
                 [ 0, 0, 1, 0.753],
                 [ 0, 0, 0,     1]])
#Intialize the initial and the final configurations of the cube. 
T_isc = np.array([[ 1, 0, 0,     1],
                  [ 0, 1, 0,     0],
                  [ 0, 0, 1, 0.025],
                  [ 0, 0, 0,     1]])

T_dsc = np.array([[ 0, 1, 0,     0],
                  [-1, 0, 0,    -1],
                  [ 0, 0, 1, 0.025],
                  [ 0, 0, 0,     1]])

#The end effector grasp frame configuration relative to the cube. 
# roty = np.array([[cos(0.785), 0, sin(0.785)],
#                  [0, 1, 0],
#                  [-sin(0.785), 0, cos(0.785)]])
T_gce = np.array([[ 0, 0, 1, 0],
                  [ 0, 1, 0, 0],
                  [-1, 0, 0, 0],
                  [ 0, 0, 0, 1]])
# T_gce[0:3,0:3] = roty @ T_gce[0:3,0:3]

#The end effector standoff frame configuration relative to the cube.
T_sce = np.array([[ 0, 0, 1,   0],
                  [ 0, 1, 0,   0],
                  [-1, 0, 0, 0.1],
                  [ 0, 0, 0,   1]])
# T_sce[0:3,0:3] = roty @ T_sce[0:3,0:3]

snip = TrajectoryGenerator(T_se, T_isc, T_dsc, T_gce, T_sce, 64)